# Experiment 03: Going deeper

In this experiment we will study how different band configurations affect the training process and end results.

### Experiment variations:

- E03-4: The 4 MS WorldView-2 bands that are also available in GeoEye-1 (MS1 array RGB+NIR1)
 - In this variation we will also validate performance on the GeoEye-1 validation set
- E03-3: Only the 3 RGB bands from WorldView-2
 - Also validated on the GeoEye-1 validation set
 
### The notebook is divided into the following main sections:
1. Imports and configuration parameters
2. Tile generation (sampling of tiles from the satellite images)
3. Tile input pipelines (`tf.dataset` objects reading tiles from disk)
4. Building of models
5. Pretraining with L1 loss
6. Build the full ESRGAN model
7. GAN-training with L1 + Percep + GAN loss
8. Inspection of results

Training history is logged with TensorBoard.

## 1. Imports and configuration parameters

In [1]:
from modules.helpers import *
from modules.tile_generator import *
from modules.matlab_metrics import *
from modules.image_utils import *
from modules.tile_input_pipeline import *
from modules.models import *
from modules.evaluation import *

from modules.logging import *
from modules.train import *

import time

# Check GPUs and enable dynamic GPU memory use:",
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            # Prevent TensorFlow from allocating all memory of all GPUs:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [2]:
### MAIN SETTINGS ###############################################################################################
EXPERIMENT_NAMES = ['e03-8', 'e03-6', 'e03-4', 'e03-3']

# Select experiment variation to be run in THIS notebook:
EXPERIMENT = EXPERIMENT_NAMES[3]

# Turn on and off certain time consuming processes in the notebook:
GENERATE_TILES = False   # This should only be done once in experiment 01. All variations will read from the same
TILE_DENSITY_MAPS = False  # Loops through all tiles and compute density maps of where tiles have been sampled
CALCULATE_STATS = False  # Loops through all tiles and calculate mean and sd. Used for scaling
PRE_BUILD = True          # Step 1 of the training process
PRETRAIN = False          # Step 1 of the training process
GAN_BUILD = True          # Step 1 of the training process
GAN_TRAIN = True         # Step 2 of the training process
PRE_EVALUATE_LAST = False
GAN_EVALUATE_LAST = False
PRE_EVALUATE_HISTORY = False
GAN_EVALUATE_HISTORY = False

# Load metadata dataframe "meta" from repository root. 
# This dataframe keeops track of images and is used and updated throughout the notebook
meta = load_meta_pickle_csv('.', 'metadata_df', from_pickle=True)
#################################################################################################################

### PATHS #######################################################################################################
DATA_PATH = 'data/toulon-laspezia'
DATA_PATH_TILES = 'data/toulon-laspezia-tiles/e03'
DATA_PATH_TILES_P = {'train': DATA_PATH_TILES + '/train', 
                     'val': DATA_PATH_TILES + '/val', 
                     'test': DATA_PATH_TILES + '/test'}
LOGS_DIR = 'logs/' # Path to tensorboard logs and model checkpoint saves
LOGS_EXP_DIR = LOGS_DIR + EXPERIMENT
#################################################################################################################

### TILE GENERATION #############################################################################################
SENSORS_GENERATE = ['WV02', 'GE01']
AREAS_GENERATE = ['La_Spezia', 'Toulon']
meta = subset_by_areas_sensor(meta, areas=AREAS_GENERATE, sensors=SENSORS_GENERATE)
print('Sensors to generate tiles from:', SENSORS_GENERATE)
print('Areas to generate tiles from:', AREAS_GENERATE)

# Count images in partitions (train/val/test):
N_IMAGES_TOTAL = count_images(meta)
N_IMAGES = {'train': count_images_in_partition(meta, 'train'), 
            'val': count_images_in_partition(meta, 'val'), 
            'test': count_images_in_partition(meta, 'test')}
assert N_IMAGES_TOTAL == sum(N_IMAGES.values())  # Verify that different ways of counting adds up
print('Number of images in partitions', N_IMAGES)
print('Total number of images:', N_IMAGES_TOTAL)

TILES_PER_M2 = {'train': 2.0, 
                'val': 2.0, 
                'test': 2.0}

# Settings for whether to send every tile generated through a sea and cloud classifier
# This is useful if images consist of a lot of sea and clouds and you want to reduce the number of tiles
# with such monotone and less meaningful content. Classifier is trained on 2500 labeled tiles of various sizes
# where only tiles COMPLETELY covered by sea and/or clouds have been labelled "cloud/sea". 
# Validation accuracy around 0.95
CLOUD_SEA_REMOVAL = True
CLOUD_SEA_WEIGHTS_PATH = 'models/cloud-sea-classifier/cloudsea-effb0-augm-bicubic-pan-0.0005--200-0.127841.h5'
# Cutoff at inference time. Tiles with (quasi)-prob higher than cutoff will be classified as cloud and or sea:
CLOUD_SEA_PRED_CUTOFF = 0.95
# Setting to keep a certain proportion of cloud/sea tiles through the filter:
CLOUD_SEA_KEEP_RATE = 0.10

# GE01 images has some slight variations in resolution 0.5 +-0.05 m per pixel while WV02 is fixed at 0.5m
# Setting this to True will resize to as close as possible to 0.5m
# Not used in this notebook, but function is ready for use in module tile_generator.py
RESIZE_TO_PIXEL_SIZE = False
if RESIZE_TO_PIXEL_SIZE:
    RESIZE_RESAMPLING_METHOD = 'nearest'  # 'nearest', 'bicubic', 'bilinear'
    NEW_PIXEL_SIZE_PAN = 0.5
    RESIZE_DIR = DATA_PATH + '-resized'
#################################################################################################################

### SENSORS AND AREA EXPERIMENT SELECTION #######################################################################
# Sensors used in which experiment variation
SENSORS_EXP = {'e03-8': {'train': 'WV02', 'val': ['WV02'], 'test': ['WV02']}, 
               'e03-6': {'train': 'WV02', 'val': ['WV02'], 'test': ['WV02']}, 
               'e03-4': {'train': 'WV02', 'val': ['WV02', 'GE01'], 'test': ['WV02', 'GE01']}, 
               'e03-3': {'train': 'WV02', 'val': ['WV02', 'GE01'], 'test': ['WV02', 'GE01']}}
SENSORS = SENSORS_EXP[EXPERIMENT]

# Areas used in which experiment variation
AREAS_EXP = {'e03-8': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}, 
             'e03-6': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}, 
             'e03-4': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}, 
             'e03-3': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}}
AREAS = AREAS_EXP[EXPERIMENT]
#################################################################################################################

### TILE DIMENSIONS #############################################################################################
# Note larger size of val and test. This is needed for sensible calculation of Ma, NIQE and PI calculation
SR_FACTOR = 4
MS_SIZE = {'train': 32, 'val': 128, 'test': 128}
PAN_SIZE = {'train': MS_SIZE['train'] * SR_FACTOR, 
            'val': MS_SIZE['val'] * SR_FACTOR, 
            'test': MS_SIZE['test'] * SR_FACTOR}
print('MS (LR) tile size:', MS_SIZE)
print('PAN (HR) tile size:', PAN_SIZE)
print('SR factor:', SR_FACTOR)
#################################################################################################################

### BAND (CHANNEL) CONFIGURATIONS ###############################################################################
# This is the essence of experiment 01
# Selection of bands is done in the tile input pipeline

# Selecting bands from the 8 bands of WV02:
WV02_FULL_BAND_CONFIG = get_sensor_bands('WV02', meta)
WV02_EXP_BAND_CONFIGS = {'e03-8': WV02_FULL_BAND_CONFIG,                          # 8 (all) bands
                        'e03-6': {k:v for (k,v) in WV02_FULL_BAND_CONFIG.items()  # 6 bands (BGYR+RE+NIR)
                                  if k not in ['Coastal', 'NIR2']}, 
                        'e03-4': {k:v for (k,v) in WV02_FULL_BAND_CONFIG.items()  # 4 bands (BGR+NIR)
                                  if k in ['Blue', 'Green', 'Red', 'NIR']},
                        'e03-3': {k:v for (k,v) in WV02_FULL_BAND_CONFIG.items()  # 3 bands (BGR)
                                  if k in ['Blue', 'Green', 'Red']}}
MS_BANDS_WV02_CONFIG = WV02_EXP_BAND_CONFIGS[EXPERIMENT]
if EXPERIMENT == 'e03-8':
    # We set this to 'all' in order to not pass e01-8 tiles through a band selection function (no reason to)
    MS_BANDS_WV02_IDXS = 'all' 
else:
    # For the other experiment variations we need lists of indices of the bands to be selected
    MS_BANDS_WV02_IDXS = list(MS_BANDS_WV02_CONFIG.values())

N_MS_BANDS = len(MS_BANDS_WV02_CONFIG.values()) # The number of MS bands in this experiment variation

# Selecting bands from the 4 bands of GE01:
GE01_FULL_BAND_CONFIG = get_sensor_bands('GE01', meta)                            
GE01_EXP_BAND_CONFIGS = {'e03-8': {None: None},                                   # not enough bands in GE01
                        'e03-6': {None: None},                                    # not enough bands in GE01
                        'e03-4': GE01_FULL_BAND_CONFIG,                           # 4 (all) bands (BGR+NIR)
                        'e03-3': {k:v for (k,v) in GE01_FULL_BAND_CONFIG.items()  # 3 bands (BGR)
                                  if k not in ['NIR']}}
MS_BANDS_GE01_CONFIG = GE01_EXP_BAND_CONFIGS[EXPERIMENT]
if EXPERIMENT == 'e03-4':
    MS_BANDS_GE01_IDXS = 'all'
else:
    MS_BANDS_GE01_IDXS = list(MS_BANDS_GE01_CONFIG.values())
print('MS (LR) Band Config WV02:', MS_BANDS_WV02_CONFIG)
print('MS (LR) Band Config GE01:', MS_BANDS_GE01_CONFIG)

N_PAN_BANDS = 1 # Obviously only 1 panchromatic band
#################################################################################################################

### MODEL PARAMETERS ############################################################################################
BATCH_SIZE = {'train': 16, 'val': 8, 'test': 8}
print('Batch sizes:', BATCH_SIZE)

# RRDB Generator Model parameters 
N_BLOCKS = 23 # Deeper means potential to capture more complex relationships, at the cost of training time
N_FILTERS = 64 # Baseline setting that is not tinkered with in this repository
#################################################################################################################

### PRETRAINING SETTINGS ########################################################################################
PRE_EPOCHS = 400
PRE_TRAIN_STEPS = 1000  # per epoch
PRE_VAL_STEPS = 0     # per epoch
print('Pretraining - Total steps:', PRE_EPOCHS * PRE_TRAIN_STEPS)

# Number of batches to save every epoch in TensorBoard
TRAIN_N_BATCHES_SAVE = 1
VAL_N_BATCHES_SAVE = 1

# Optimizer settings:
PRETRAIN_LOSS = 'l1'    # Official
PRETRAIN_LR = 5e-5      # Tuned and found stable for this particular experiment
#PRETRAIN_LR = 0.0002   # Official
PRETRAIN_BETA_1 = 0.9   # Official
PRETRAIN_BETA_2 = 0.999 # Official
# Note: Official implementation also uses stepwise learning rate scheduler. 
# This is avoided here as it is deemed not central to the experiment to "squeeze" out last performance and it 
# complicates comparisons between experiment variations
#################################################################################################################

### GAN TRAINING SETTINGS #######################################################################################
GAN_EPOCHS = 400
GAN_TRAIN_STEPS = 1000
GAN_VAL_STEPS = 0
# Proportion of val batches that will go through ma and niqe metric calculation
# MA_NIQE_PROPORTION = 0.04  # The calculation is very time consuming
MA_NIQE_PROPORTION = 1  # The calculation is very time consuming
print('GAN training - Total steps:', GAN_EPOCHS * GAN_TRAIN_STEPS)

# Weights for each loss in the composite loss function
G_LOSS_PIXEL_W = 0.01       # Official
G_LOSS_PERCEP_W = 1.0       # Official
G_LOSS_GENERATOR_W = 0.005  # Official

# Optimizer settings:
#GAN_G_LR = 1e-4 # Official
#GAN_D_LR = 1e-4 # Official
GAN_G_LR = 2e-5
GAN_D_LR = 2e-5
G_BETA_1, D_BETA_1 = 0.9, 0.9      # Official
G_BETA_2, D_BETA_2 = 0.999, 0.999  # Official
# Note: Official implementation also uses stepwise learning rate scheduler. 
# This is avoided here as it is deemed not central to the experiment to "squeeze" out last performance and it 
# complicates comparisons between experiment variations

# Path to the pretraining weights that is the starting point of GAN training:
PRETRAIN_WEIGHTS_DIRS = {'e03-3': LOGS_EXP_DIR + '/models/' + 'e03-3-pre_20210306-143849/'
                        }
PRETRAIN_WEIGHTS_DIR = PRETRAIN_WEIGHTS_DIRS[EXPERIMENT]
PRETRAIN_WEIGHTS_PATH = PRETRAIN_WEIGHTS_DIR + EXPERIMENT + '-pre-400.h5'

# Path to the gan-training weights that will be 
GAN_WEIGHTS_DIRS = {'e03-3': LOGS_EXP_DIR + '/models/' + 'e03-3-gan_20210310-092747/'
                   }
GAN_WEIGHTS_DIR = GAN_WEIGHTS_DIRS[EXPERIMENT]
GAN_WEIGHTS_PATH = GAN_WEIGHTS_DIR + EXPERIMENT + '-gan-G-399.h5'
#################################################################################################################

### MATLAB METRICS ##############################################################################################
# Calculate Ma, NIQE and Perceptual Index (PI) metrics on the validation set(s) during GAN training:
# PI was metric used in PIRM2018 competition https://github.com/roimehrez/PIRM2018
METRIC_MA = False
METRIC_NIQE = False
if METRIC_MA and METRIC_NIQE:
    METRIC_PI = True
else:
    METRIC_PI = False

# The number of pixels to be shaved off the border of the tile before calculating Ma/NIQE/PI (ignore border effects)
SHAVE_WIDTH = 4 # Official (as used in PIRM2018 evaluation)
# Ma/NIQE/PI calculation is done with official matlab repositories through MATLAB Engine API for Python
MATLAB_PATH = 'modules/matlab' # path to repositories
#################################################################################################################

### EVALUTAION ##################################################################################################
if PRE_EVALUATE_LAST or GAN_EVALUATE_LAST:
    METRIC_MA = False
    METRIC_NIQE = True
    if METRIC_MA and METRIC_NIQE:
        METRIC_PI = True
    else:
        METRIC_PI = False
        
if PRE_EVALUATE_HISTORY and GAN_EVALUATE_HISTORY:
    raise ValueError

EVAL_STEPS_PER_EPOCH = 2000
EVAL_N_EPOCHS = 400
EVAL_SENSOR = 'WV02'
EVAL_PER_IMAGE = True
    
if PRE_EVALUATE_HISTORY:
    EVAL_WEIGHTS_DIR = PRETRAIN_WEIGHTS_DIR
    EVAL_FIRST_STEP = 1
    EVAL_PREFIX = EXPERIMENT + '-pre-'
elif GAN_EVALUATE_HISTORY:
    EVAL_WEIGHTS_DIR = GAN_WEIGHTS_DIR
    EVAL_FIRST_STEP = 0
    EVAL_PREFIX = EXPERIMENT + '-gan-'
    
print('MATLAB Metrics:')
print('Ma:', METRIC_MA)
print('NIQE:', METRIC_NIQE)
print('Perceptual Index (PI):', METRIC_PI)

Sensors to generate tiles from: ['WV02', 'GE01']
Areas to generate tiles from: ['La_Spezia', 'Toulon']
Number of images in partitions {'train': 22, 'val': 19, 'test': 21}
Total number of images: 62
MS (LR) tile size: {'train': 32, 'val': 128, 'test': 128}
PAN (HR) tile size: {'train': 128, 'val': 512, 'test': 512}
SR factor: 4
MS (LR) Band Config WV02: {'Blue': 1, 'Green': 2, 'Red': 4}
MS (LR) Band Config GE01: {'Blue': 0, 'Green': 1, 'Red': 2}
Batch sizes: {'train': 16, 'val': 8, 'test': 8}
Pretraining - Total steps: 400000
GAN training - Total steps: 400000
MATLAB Metrics:
Ma: False
NIQE: False
Perceptual Index (PI): False


## 2. Tile generation

### 2.1 Image resizing

Function `resize_sat_img_to_new_pixel_size` available in `modules.tile_generator`. Not used in this notebook

### 2.2 Tile allocation

We allocate `n_tiles` to each satellite image in proportion to the area covered by the satellite image. We adjust `n_tiles` by the argument `tiles_per_m2`. If `tiles_per_m2=1.0` then `n_tiles` is set deterministically to a value so that a square meter of satellite image is expected to be covered by `1.0` tile.

In [3]:
if GENERATE_TILES:
    meta = allocate_tiles_by_expected(meta, 
                                      override_pan_pixel_size=RESIZE_TO_PIXEL_SIZE,
                                      by_partition=True, 
                                      tiles_per_m2_train_val_test=(TILES_PER_M2['train'], 
                                                                   TILES_PER_M2['val'], 
                                                                   TILES_PER_M2['test']),
                                      pan_tile_size_train_val_test=(PAN_SIZE['train'], 
                                                                    PAN_SIZE['val'], 
                                                                    PAN_SIZE['test']),
                                      new_column_name='n_tiles')
else:
    # Load meta dataframe that was updated at tile generation time
    meta = load_meta_pickle_csv(DATA_PATH_TILES, 'metadata_tile_allocation', from_pickle=True)

n_tiles = {'train': count_tiles_in_partition(meta, 'train'),
           'val': count_tiles_in_partition(meta, 'val'), 
           'test':  count_tiles_in_partition(meta, 'test')}
n_tiles_total = count_tiles(meta)
assert n_tiles_total == sum(n_tiles.values())
print('Number of tiles per partition:')
print(n_tiles)
print('Total number of tiles:', n_tiles_total)

Number of tiles per partition:
{'train': 129280, 'val': 8066, 'test': 9222}
Total number of tiles: 146568


### 2.3 Tile generation to disk

In [4]:
if GENERATE_TILES:
    meta = generate_all_tiles(meta, 
                              save_dir=DATA_PATH_TILES, 
                              sr_factor=SR_FACTOR, 
                              by_partition=True,
                              ms_tile_size_train_val_test=(MS_SIZE['train'], MS_SIZE['val'], MS_SIZE['test']), 
                              cloud_sea_removal=CLOUD_SEA_REMOVAL, 
                              cloud_sea_weights_path=CLOUD_SEA_WEIGHTS_PATH, 
                              cloud_sea_pred_cutoff=CLOUD_SEA_PRED_CUTOFF,
                              cloud_sea_keep_rate=CLOUD_SEA_KEEP_RATE,
                              save_meta_to_disk=True)

In [5]:
if TILE_DENSITY_MAPS:
    for row in meta.iterrows():
        img_uid = row[0]
        density = tile_density_map(DATA_PATH_TILES, 
                                   row[1], 
                                   pan_or_ms='pan',
                                   density_dtype='uint8',
                                   write_to_disk=True,
                                   write_dir=DATA_PATH_TILES + '/density-maps', 
                                   write_filename=img_uid)
    # Plot last density
    plt.imshow(density)

In [6]:
if CALCULATE_STATS:
    train_tiles_mean, train_tiles_sd = mean_sd_of_train_tiles(DATA_PATH_TILES, 
                                                              sample_proportion=1.0, 
                                                              write_json=True)
else:
    train_tiles_mean, train_tiles_sd = read_mean_sd_json(DATA_PATH_TILES)

Loaded mean 339.8 and sd 128.3 from json file @ data/toulon-laspezia-tiles/e03/train_mean_sd.json


## 3. Data input pipeline from disk

### 3.1 Training set

In [7]:
SHUFFLE_BUFFER_SIZE = {'train': n_tiles['train'],  # 100
                       'val': n_tiles['val'],  # 100
                       'test': n_tiles['test']}  # 100

train_val_test = 'train'
sensor = SENSORS[train_val_test]
ds_train = {sensor: GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                   batch_size=BATCH_SIZE[train_val_test], 
                                   ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                   pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                   sensor=sensor,
                                   band_selection=MS_BANDS_WV02_IDXS, 
                                   mean_correction=train_tiles_mean,
                                   cache_memory=True,
                                   cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-'
                                                    + train_val_test + '-' + sensor + '_cache'), 
                                   repeat=True, 
                                   shuffle=True, 
                                   shuffle_buffer_size=SHUFFLE_BUFFER_SIZE[train_val_test])
           }
# Getting the scaled output range from the scaler. Needed to calculate PSNR and SSIM:
scaled_range = ds_train[sensor].get_scaler_output_range(print_ranges=True)

# Returning the actual tf.data.dataset object:
ds_train[sensor] = ds_train[sensor].get_dataset()
print(ds_train.keys())

Scaler ranges:
Input (uint) min, max: 0 2047
Input (uint) range: 2048
Output (float) range 1.1996114807902925
Output (float) min, max: -0.19902573299693777 1.0
dict_keys(['WV02'])


### 3.2 Validation set

In [8]:
# Validation set can have several sensors and is organized in a dictionary
# structure: ds_val = {sensor: dataset} ... ex: ds_val = {'WV02': dataset_with_only_WV02_images}
train_val_test = 'val'
ds_val = {}
for sensor in SENSORS[train_val_test]:
    if sensor == 'WV02':
        band_indices = MS_BANDS_WV02_IDXS
    elif sensor == 'GE01':
        band_indices = MS_BANDS_GE01_IDXS
    ds_val[sensor] = GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                    batch_size=BATCH_SIZE[train_val_test], 
                                    ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                    pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                    sensor=sensor,
                                    band_selection=band_indices, 
                                    mean_correction=train_tiles_mean,
                                    cache_memory=True,
                                    cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-'
                                                   + train_val_test + '-' + sensor + '_cache'), 
                                    repeat=True, 
                                    shuffle=True, 
                                    shuffle_buffer_size=SHUFFLE_BUFFER_SIZE[train_val_test])
    ds_val[sensor] = ds_val[sensor].get_dataset()
print(ds_val.keys())

dict_keys(['WV02', 'GE01'])


## 4. Build preliminary models

### 4.1 Bicubic baseline model

In [9]:
bicubic = build_deterministic_sr_model(upsample_factor=SR_FACTOR,
                                       resize_method='bicubic',
                                       loss='mean_absolute_error',
                                       metrics=('PSNR', 'SSIM'),
                                       scaled_range=scaled_range)

### 4.2 ESRGAN Generator model (pretrain version)

In [10]:
if PRE_BUILD:
    pretrain_model =  build_generator(pretrain_or_gan='pretrain', 
                                      pretrain_learning_rate=PRETRAIN_LR, 
                                      pretrain_loss_l1_l2=PRETRAIN_LOSS,
                                      pretrain_beta_1=PRETRAIN_BETA_1, 
                                      pretrain_beta_2=PRETRAIN_BETA_2, 
                                      pretrain_metrics=('PSNR', 'SSIM'),
                                      scaled_range=scaled_range, 
                                      n_channels_in=N_MS_BANDS, 
                                      n_channels_out=N_PAN_BANDS, 
                                      height_width_in=None,  # None will make network image size agnostic
                                      n_filters=N_FILTERS, 
                                      n_blocks=N_BLOCKS)
    # pretrain_model.summary()

## 5. Pretraining with L1 loss

In [11]:
if PRETRAIN:
    history = pretrain_esrgan(generator=pretrain_model,
                              ds_train_dict=ds_train,
                              epochs=PRE_EPOCHS,
                              steps_per_epoch=PRE_TRAIN_STEPS,
                              initial_epoch=0,
                              validate=True,
                              ds_val_dict=ds_val,
                              val_steps=PRE_VAL_STEPS,
                              model_name=EXPERIMENT + '-pre',
                              tag=EXPERIMENT,
                              log_tensorboard=True,
                              tensorboard_logs_dir=LOGS_EXP_DIR + '/tb',
                              save_models=True,
                              models_save_dir=LOGS_EXP_DIR + '/models',
                              save_weights_only=True,
                              log_train_images=True,
                              n_train_image_batches=TRAIN_N_BATCHES_SAVE,
                              log_val_images=True,
                              n_val_image_batches=VAL_N_BATCHES_SAVE)

## 6. Build the full ESRGAN Model

In [12]:
if GAN_BUILD:
    gan_model = build_esrgan_model(PRETRAIN_WEIGHTS_PATH,
                                   n_channels_in=N_MS_BANDS, 
                                   n_channels_out=N_PAN_BANDS, 
                                   n_filters=N_FILTERS, 
                                   n_blocks=N_BLOCKS, 
                                   pan_shape=(PAN_SIZE['train'], PAN_SIZE['train'], N_PAN_BANDS),
                                   G_lr=GAN_G_LR, 
                                   D_lr=GAN_D_LR, 
                                   G_beta_1=G_BETA_1, 
                                   G_beta_2=G_BETA_2, 
                                   D_beta_1=D_BETA_1, 
                                   D_beta_2=D_BETA_2,
                                   G_loss_pixel_w=G_LOSS_PIXEL_W, 
                                   G_loss_pixel_l1_l2='l1',
                                   G_loss_percep_w=G_LOSS_PERCEP_W, 
                                   G_loss_percep_l1_l2='l1', 
                                   G_loss_percep_layer=54,
                                   G_loss_percep_before_act=True,
                                   G_loss_generator_w=G_LOSS_GENERATOR_W,
                                   metric_reg=False, 
                                   metric_ma=METRIC_MA, 
                                   metric_niqe=METRIC_NIQE, 
                                   ma_niqe_proportion=MA_NIQE_PROPORTION,
                                   matlab_wd_path='modules/matlab',
                                   scale_mean=train_tiles_mean, 
                                   scaled_range=scaled_range, 
                                   shave_width=SHAVE_WIDTH)

## 7. GAN training

In [13]:
if GAN_TRAIN:
    history = gan_train_esrgan(esrgan_model=gan_model,
                               ds_train_dict=ds_train,
                               epochs=GAN_EPOCHS,
                               steps_per_epoch=GAN_TRAIN_STEPS,
                               initial_epoch=0,
                               validate=True,
                               ds_val_dict=ds_val,
                               val_steps=GAN_VAL_STEPS,
                               model_name=EXPERIMENT + '-gan',
                               tag=EXPERIMENT,
                               log_tensorboard=True,
                               tensorboard_logs_dir=LOGS_EXP_DIR + '/tb',
                               save_models=True,
                               models_save_dir=LOGS_EXP_DIR + '/models',
                               save_weights_only=True,
                               log_train_images=True,
                               n_train_image_batches=TRAIN_N_BATCHES_SAVE,
                               log_val_images=True,
                               n_val_image_batches=VAL_N_BATCHES_SAVE)

16 images from train-WV02 will be logged at each epoch
8 images from val-WV02 will be logged at each epoch
8 images from val-GE01 will be logged at each epoch
Callbacks: [<modules.logging.SimpleEsrganCheckpoint object at 0x000001EDAE1A3FC8>, <tensorflow.python.keras.callbacks.TensorBoard object at 0x000001ED4DBA4E48>, <modules.logging.MultipleValSetsCallback object at 0x000001EDADBB97C8>, <modules.logging.LrHrSrImageCallback object at 0x000001EDADBB1CC8>, <modules.logging.LrHrSrImageCallback object at 0x000001EDAE1A6988>]
Epoch 1/400
1000/1000 [==============================] - 605s 605ms/step - G_pixel_loss: 1.1823e-04 - G_perceptual_loss: 1.9980 - G_generator_loss: 0.0200 - G_loss_total: 2.0182 - D_loss_total: 0.1945 - PSNR: 38.2803 - SSIM: 0.8973
Epoch 2/400
1000/1000 [==============================] - 606s 606ms/step - G_pixel_loss: 1.2666e-04 - G_perceptual_loss: 1.9608 - G_generator_loss: 0.0336 - G_loss_total: 1.9945 - D_loss_total: 0.0423 - PSNR: 37.4812 - SSIM: 0.8934
Epoch 3/

1000/1000 [==============================] - 604s 604ms/step - G_pixel_loss: 1.1985e-04 - G_perceptual_loss: 1.7116 - G_generator_loss: 0.0431 - G_loss_total: 1.7548 - D_loss_total: 0.0524 - PSNR: 38.0588 - SSIM: 0.8860
Epoch 69/400
1000/1000 [==============================] - 605s 605ms/step - G_pixel_loss: 1.2107e-04 - G_perceptual_loss: 1.7281 - G_generator_loss: 0.0443 - G_loss_total: 1.7726 - D_loss_total: 0.0545 - PSNR: 38.0104 - SSIM: 0.8852
Epoch 70/400
1000/1000 [==============================] - 606s 606ms/step - G_pixel_loss: 1.2159e-04 - G_perceptual_loss: 1.7281 - G_generator_loss: 0.0413 - G_loss_total: 1.7694 - D_loss_total: 0.0643 - PSNR: 38.0085 - SSIM: 0.8855
Epoch 71/400
1000/1000 [==============================] - 609s 609ms/step - G_pixel_loss: 1.2050e-04 - G_perceptual_loss: 1.7197 - G_generator_loss: 0.0403 - G_loss_total: 1.7601 - D_loss_total: 0.0607 - PSNR: 38.0557 - SSIM: 0.8858
Epoch 72/400
1000/1000 [==============================] - 610s 610ms/step - G_pix

1000/1000 [==============================] - 603s 603ms/step - G_pixel_loss: 1.2048e-04 - G_perceptual_loss: 1.5904 - G_generator_loss: 0.0513 - G_loss_total: 1.6418 - D_loss_total: 0.0412 - PSNR: 38.0831 - SSIM: 0.8850
Epoch 139/400
1000/1000 [==============================] - 603s 603ms/step - G_pixel_loss: 1.2053e-04 - G_perceptual_loss: 1.5870 - G_generator_loss: 0.0510 - G_loss_total: 1.6381 - D_loss_total: 0.0427 - PSNR: 38.1031 - SSIM: 0.8855
Epoch 140/400
1000/1000 [==============================] - 603s 603ms/step - G_pixel_loss: 1.2041e-04 - G_perceptual_loss: 1.5797 - G_generator_loss: 0.0499 - G_loss_total: 1.6297 - D_loss_total: 0.0448 - PSNR: 38.0554 - SSIM: 0.8845
Epoch 141/400
1000/1000 [==============================] - 603s 603ms/step - G_pixel_loss: 1.2075e-04 - G_perceptual_loss: 1.5868 - G_generator_loss: 0.0491 - G_loss_total: 1.6361 - D_loss_total: 0.0390 - PSNR: 38.0364 - SSIM: 0.8847
Epoch 142/400
1000/1000 [==============================] - 603s 603ms/step - G

1000/1000 [==============================] - 606s 606ms/step - G_pixel_loss: 1.2101e-04 - G_perceptual_loss: 1.5062 - G_generator_loss: 0.0631 - G_loss_total: 1.5694 - D_loss_total: 0.0265 - PSNR: 38.0815 - SSIM: 0.8842
Epoch 209/400
1000/1000 [==============================] - 605s 605ms/step - G_pixel_loss: 1.2124e-04 - G_perceptual_loss: 1.5172 - G_generator_loss: 0.0600 - G_loss_total: 1.5773 - D_loss_total: 0.0210 - PSNR: 38.0685 - SSIM: 0.8839
Epoch 210/400
1000/1000 [==============================] - 604s 604ms/step - G_pixel_loss: 1.2049e-04 - G_perceptual_loss: 1.5178 - G_generator_loss: 0.0592 - G_loss_total: 1.5771 - D_loss_total: 0.0384 - PSNR: 38.1150 - SSIM: 0.8850
Epoch 211/400
1000/1000 [==============================] - 604s 604ms/step - G_pixel_loss: 1.2167e-04 - G_perceptual_loss: 1.4961 - G_generator_loss: 0.0542 - G_loss_total: 1.5504 - D_loss_total: 0.0334 - PSNR: 38.0208 - SSIM: 0.8834
Epoch 212/400
1000/1000 [==============================] - 604s 604ms/step - G

1000/1000 [==============================] - 603s 603ms/step - G_pixel_loss: 1.2034e-04 - G_perceptual_loss: 1.4246 - G_generator_loss: 0.0561 - G_loss_total: 1.4808 - D_loss_total: 0.0311 - PSNR: 38.0764 - SSIM: 0.8849
Epoch 279/400
1000/1000 [==============================] - 603s 603ms/step - G_pixel_loss: 1.2187e-04 - G_perceptual_loss: 1.4391 - G_generator_loss: 0.0643 - G_loss_total: 1.5036 - D_loss_total: 0.0166 - PSNR: 38.0015 - SSIM: 0.8832
Epoch 280/400
1000/1000 [==============================] - 603s 603ms/step - G_pixel_loss: 1.2235e-04 - G_perceptual_loss: 1.4418 - G_generator_loss: 0.0667 - G_loss_total: 1.5087 - D_loss_total: 0.0283 - PSNR: 37.8993 - SSIM: 0.8834
Epoch 281/400
1000/1000 [==============================] - 603s 603ms/step - G_pixel_loss: 1.2205e-04 - G_perceptual_loss: 1.4417 - G_generator_loss: 0.0621 - G_loss_total: 1.5040 - D_loss_total: 0.0210 - PSNR: 37.9456 - SSIM: 0.8830
Epoch 282/400
1000/1000 [==============================] - 603s 603ms/step - G

1000/1000 [==============================] - 604s 604ms/step - G_pixel_loss: 1.2053e-04 - G_perceptual_loss: 1.3768 - G_generator_loss: 0.0702 - G_loss_total: 1.4471 - D_loss_total: 0.0096 - PSNR: 38.0644 - SSIM: 0.8852
Epoch 349/400
1000/1000 [==============================] - 603s 603ms/step - G_pixel_loss: 1.2244e-04 - G_perceptual_loss: 1.3786 - G_generator_loss: 0.0682 - G_loss_total: 1.4469 - D_loss_total: 0.0207 - PSNR: 37.8607 - SSIM: 0.8821
Epoch 350/400
1000/1000 [==============================] - 603s 603ms/step - G_pixel_loss: 1.2057e-04 - G_perceptual_loss: 1.3696 - G_generator_loss: 0.0603 - G_loss_total: 1.4301 - D_loss_total: 0.0316 - PSNR: 38.0874 - SSIM: 0.8842
Epoch 351/400
1000/1000 [==============================] - 603s 603ms/step - G_pixel_loss: 1.2247e-04 - G_perceptual_loss: 1.3818 - G_generator_loss: 0.0665 - G_loss_total: 1.4485 - D_loss_total: 0.0129 - PSNR: 37.9556 - SSIM: 0.8835
Epoch 352/400
1000/1000 [==============================] - 603s 603ms/step - G

## 8. Evaluation

### 8.1 Data input pipelines for final evaluation

The pipeline is modified to include the file paths of the tiles/patches so that it is possible to log performance metrics for individual files and by extension for individual satellite images.

#### 8.1.1 Validation set

In [14]:
# Validation set can have several sensors and is organized in a dictionary
# structure: ds_val = {sensor: dataset} ... ex: ds_val = {'WV02': dataset_with_only_WV02_images}
train_val_test = 'val'
ds_val = {}
for sensor in SENSORS[train_val_test]:
    if sensor == 'WV02':
        band_indices = MS_BANDS_WV02_IDXS
    elif sensor == 'GE01':
        band_indices = MS_BANDS_GE01_IDXS
    ds_val[sensor] = GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                    batch_size=BATCH_SIZE[train_val_test], 
                                    ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                    pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                    sensor=sensor,
                                    band_selection=band_indices, 
                                    mean_correction=train_tiles_mean,
                                    cache_memory=False,
                                    cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-'
                                                   + train_val_test + '-' + sensor + '_filepath_cache'), 
                                    repeat=False, 
                                    shuffle=False, 
                                    shuffle_buffer_size=0, #SHUFFLE_BUFFER_SIZE[train_val_test], 
                                    include_file_paths=True)
    ds_val[sensor] = ds_val[sensor].get_dataset()
print(ds_val.keys())

dict_keys(['WV02', 'GE01'])


#### 8.1.2 Test set

In [15]:
train_val_test = 'test'
ds_test = {}
for sensor in SENSORS[train_val_test]:
    if sensor == 'WV02':
        band_indices = MS_BANDS_WV02_IDXS
    elif sensor == 'GE01':
        band_indices = MS_BANDS_GE01_IDXS
    ds_test[sensor] = GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                     batch_size=BATCH_SIZE[train_val_test], 
                                     ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                     pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                     sensor=sensor,
                                     band_selection=band_indices, 
                                     mean_correction=train_tiles_mean,
                                     cache_memory=False,
                                     cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-'
                                                    + train_val_test + '-' + sensor + '_filepath_cache'), 
                                     repeat=False, 
                                     shuffle=False, 
                                     shuffle_buffer_size=0)
    ds_test[sensor] = ds_test[sensor].get_dataset()
print(ds_test.keys())

dict_keys(['WV02', 'GE01'])


### 8.2 Evaluate last epoch

In [16]:
val_or_test = 'val'

# Computing Ma is 100x more time consuming than anything else. It is not interesting to measure this for pretraining
if METRIC_MA:
    PRE_GAN = ['gan']
else:
    PRE_GAN = ['pre', 'gan']
    
for pre_gan in PRE_GAN:
    for sensor in SENSORS[val_or_test]:
        if sensor == 'GE01':
            band_indices = MS_BANDS_GE01_IDXS
        elif sensor == 'WV02':
            band_indices = MS_BANDS_WV02_IDXS
        if pre_gan == 'pre':
            gan_model.G.load_weights(PRETRAIN_WEIGHTS_PATH)
        else:
            gan_model.G.load_weights(GAN_WEIGHTS_PATH)

        start = time.time()
        results_df = esrgan_evaluate(model=gan_model, 
                                     dataset=ds_val[sensor], 
                                     steps='all', 
                                     per_image=True, 
                                     write_csv=True,
                                     csv_path=str(LOGS_EXP_DIR + '/csv/' + 'final_epoch-' 
                                                  + pre_gan + '-' + val_or_test + '-' + sensor + '.csv'), 
                                     verbose=1
                                    )
        end = time.time()
        print(str((end - start) / 60), 'minutes')

Computed 8 images in  7.904633283615112 seconds
Last image: {'G_pixel_loss': 0.00010941584332613274, 'G_perceptual_loss': 2.435647487640381, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 34.262935638427734, 'SSIM': 0.9041274785995483}
Computed 8 images in  1.8205442428588867 seconds
Last image: {'G_pixel_loss': 0.00016422531916759908, 'G_perceptual_loss': 3.548758029937744, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 31.549333572387695, 'SSIM': 0.8340572118759155}
Computed 8 images in  1.6773755550384521 seconds
Last image: {'G_pixel_loss': 0.00011429814185248688, 'G_perceptual_loss': 1.2457382678985596, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 36.9582405090332, 'SSIM': 0.894885241985321}
Computed 8 images in  1.7187840938568115 seconds
Last image: {'G_pixel_loss': 4.574274498736486e-05, 'G_perceptual_loss': 1.1143461465835571, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0,

Computed 8 images in  1.6998097896575928 seconds
Last image: {'G_pixel_loss': 4.4899315980728716e-05, 'G_perceptual_loss': 1.0207231044769287, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 40.96074295043945, 'SSIM': 0.9865916967391968}
Computed 8 images in  1.7433605194091797 seconds
Last image: {'G_pixel_loss': 9.71060580923222e-05, 'G_perceptual_loss': 1.7395555973052979, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 35.92825698852539, 'SSIM': 0.9265092015266418}
Computed 8 images in  1.7547476291656494 seconds
Last image: {'G_pixel_loss': 0.00014379387721419334, 'G_perceptual_loss': 1.8838499784469604, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 35.638797760009766, 'SSIM': 0.8183903694152832}
Computed 8 images in  1.7285213470458984 seconds
Last image: {'G_pixel_loss': 0.000122191064292565, 'G_perceptual_loss': 1.9095498323440552, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0

Computed 8 images in  1.8054180145263672 seconds
Last image: {'G_pixel_loss': 3.5585973819252104e-05, 'G_perceptual_loss': 1.0011637210845947, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 44.19208908081055, 'SSIM': 0.9845907688140869}
Computed 8 images in  1.7505619525909424 seconds
Last image: {'G_pixel_loss': 0.0001587188889971003, 'G_perceptual_loss': 3.097517251968384, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 33.086219787597656, 'SSIM': 0.7991812229156494}
Computed 8 images in  1.699697732925415 seconds
Last image: {'G_pixel_loss': 2.795574619085528e-05, 'G_perceptual_loss': 0.46990981698036194, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 51.96682357788086, 'SSIM': 0.9987155795097351}
Computed 8 images in  1.7656447887420654 seconds
Last image: {'G_pixel_loss': 0.00010373047553002834, 'G_perceptual_loss': 1.0614187717437744, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.

Computed 8 images in  1.804739236831665 seconds
Last image: {'G_pixel_loss': 3.6887671740259975e-05, 'G_perceptual_loss': 0.7618482112884521, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 48.43415451049805, 'SSIM': 0.9919188618659973}
Computed 8 images in  1.7271122932434082 seconds
Last image: {'G_pixel_loss': 0.0001680629065958783, 'G_perceptual_loss': 4.136988639831543, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 31.80590057373047, 'SSIM': 0.7703025341033936}
Computed 8 images in  1.7236275672912598 seconds
Last image: {'G_pixel_loss': 0.000136859409394674, 'G_perceptual_loss': 2.986307144165039, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 34.61737060546875, 'SSIM': 0.7921030521392822}
Computed 8 images in  1.6911933422088623 seconds
Last image: {'G_pixel_loss': 0.00021585868671536446, 'G_perceptual_loss': 3.870701789855957, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'P

Computed 8 images in  1.7145113945007324 seconds
Last image: {'G_pixel_loss': 0.000131053282530047, 'G_perceptual_loss': 1.7868874073028564, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 35.90431213378906, 'SSIM': 0.8074548244476318}
Computed 8 images in  1.7695949077606201 seconds
Last image: {'G_pixel_loss': 0.00019416313443798572, 'G_perceptual_loss': 1.191694736480713, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 31.75295639038086, 'SSIM': 0.8922107219696045}
Computed 8 images in  1.693558931350708 seconds
Last image: {'G_pixel_loss': 0.0005476507358253002, 'G_perceptual_loss': 1.4953166246414185, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 26.496517181396484, 'SSIM': 0.6375440955162048}
Computed 8 images in  1.7598474025726318 seconds
Last image: {'G_pixel_loss': 0.0005495044169947505, 'G_perceptual_loss': 0.8718600273132324, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 

...

Computed 8 images in  1.649576187133789 seconds
Last image: {'G_pixel_loss': 0.0003579764743335545, 'G_perceptual_loss': 4.842954635620117, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 26.950212478637695, 'SSIM': 0.5625519752502441}
Computed 8 images in  1.7259044647216797 seconds
Last image: {'G_pixel_loss': 0.00017784483497962356, 'G_perceptual_loss': 4.028205871582031, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 30.50564956665039, 'SSIM': 0.8464452624320984}
Computed 8 images in  1.7195007801055908 seconds
Last image: {'G_pixel_loss': 0.00038885261164978147, 'G_perceptual_loss': 5.361133575439453, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 25.398563385009766, 'SSIM': 0.7281838655471802}
Computed 8 images in  1.6950311660766602 seconds
Last image: {'G_pixel_loss': 0.00019266379240434617, 'G_perceptual_loss': 3.646854877471924, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0,

Computed 8 images in  1.7304954528808594 seconds
Last image: {'G_pixel_loss': 0.00014719454338774085, 'G_perceptual_loss': 3.5276858806610107, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 30.466421127319336, 'SSIM': 0.8907167911529541}
Computed 8 images in  1.7113723754882812 seconds
Last image: {'G_pixel_loss': 2.1905749235884286e-05, 'G_perceptual_loss': 0.55511474609375, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 51.90869140625, 'SSIM': 0.996868371963501}
Computed 8 images in  1.7559826374053955 seconds
Last image: {'G_pixel_loss': 0.00025925226509571075, 'G_perceptual_loss': 6.112897872924805, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 27.731037139892578, 'SSIM': 0.8152756094932556}
Computed 8 images in  1.7091691493988037 seconds
Last image: {'G_pixel_loss': 0.0002735304005909711, 'G_perceptual_loss': 3.778233289718628, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'P

Computed 8 images in  1.7572288513183594 seconds
Last image: {'G_pixel_loss': 0.00037334291846491396, 'G_perceptual_loss': 0.4341723322868347, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 30.12344741821289, 'SSIM': 0.9383243918418884}
Computed 8 images in  1.7251293659210205 seconds
Last image: {'G_pixel_loss': 0.0008757644100114703, 'G_perceptual_loss': 6.88824462890625, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 21.181161880493164, 'SSIM': 0.3768794536590576}
Computed 8 images in  1.741265058517456 seconds
Last image: {'G_pixel_loss': 0.0009298926452174783, 'G_perceptual_loss': 5.838910102844238, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 21.056068420410156, 'SSIM': 0.37524157762527466}
Computed 8 images in  1.7799639701843262 seconds
Last image: {'G_pixel_loss': 0.0003502141626086086, 'G_perceptual_loss': 0.4111577868461609, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0,

Computed 8 images in  1.7467358112335205 seconds
Last image: {'G_pixel_loss': 0.000581905129365623, 'G_perceptual_loss': 2.314525604248047, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 25.58258056640625, 'SSIM': 0.5405020713806152}
Computed 8 images in  1.775043249130249 seconds
Last image: {'G_pixel_loss': 0.0007394748972728848, 'G_perceptual_loss': 4.183317184448242, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 22.925926208496094, 'SSIM': 0.22916871309280396}
Computed 8 images in  1.7044425010681152 seconds
Last image: {'G_pixel_loss': 0.00047152131446637213, 'G_perceptual_loss': 1.735945701599121, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 26.96503257751465, 'SSIM': 0.7672867774963379}
Computed 8 images in  1.754915714263916 seconds
Last image: {'G_pixel_loss': 0.00048579482245258987, 'G_perceptual_loss': 0.8294820189476013, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, '

FileNotFoundError: [Errno 2] No such file or directory: 'logs\\e03-3\\csv\\final_epoch-pre-val-WV02.csv'

### 8.3 Evaluate every kth epoch

### 8.4 Comparison plots